<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Union" data-toc-modified-id="Union-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Union</a></span></li><li><span><a href="#Union.-Различный-порядок-колонок" data-toc-modified-id="Union.-Различный-порядок-колонок-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Union. Различный порядок колонок</a></span></li><li><span><a href="#Safe-union" data-toc-modified-id="Safe-union-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Safe union</a></span></li><li><span><a href="#Regexp" data-toc-modified-id="Regexp-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Regexp</a></span></li><li><span><a href="#Explode" data-toc-modified-id="Explode-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Explode</a></span></li><li><span><a href="#Map-to-columns" data-toc-modified-id="Map-to-columns-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Map to columns</a></span></li><li><span><a href="#explode_outer" data-toc-modified-id="explode_outer-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>explode_outer</a></span></li><li><span><a href="#Window-functions" data-toc-modified-id="Window-functions-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Window functions</a></span><ul class="toc-item"><li><span><a href="#Create-df-for-window-examples" data-toc-modified-id="Create-df-for-window-examples-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Create df for window examples</a></span></li><li><span><a href="#row_number,-rank,-dense_rank,-percent_rank,-ntile,-cume_dist" data-toc-modified-id="row_number,-rank,-dense_rank,-percent_rank,-ntile,-cume_dist-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>row_number, rank, dense_rank, percent_rank, ntile, cume_dist</a></span></li><li><span><a href="#lag-Window-Function" data-toc-modified-id="lag-Window-Function-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>lag Window Function</a></span></li><li><span><a href="#Window-Aggregate-Functions" data-toc-modified-id="Window-Aggregate-Functions-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span>Window Aggregate Functions</a></span></li></ul></li><li><span><a href="#UDF" data-toc-modified-id="UDF-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>UDF</a></span><ul class="toc-item"><li><span><a href="#Обработка-null" data-toc-modified-id="Обработка-null-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Обработка null</a></span></li><li><span><a href="#SQL-register-UDF" data-toc-modified-id="SQL-register-UDF-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>SQL register UDF</a></span></li><li><span><a href="#Lambda" data-toc-modified-id="Lambda-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Lambda</a></span></li><li><span><a href="#Обработка-массивов" data-toc-modified-id="Обработка-массивов-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Обработка массивов</a></span></li></ul></li></ul></div>

## Union

In [ ]:
%pyspark

from pyspark.sql import SparkSession


simpleData = [
    ("James","Sales","NY",90000,34,10000), 
    ("Michael","Sales","NY",86000,56,20000), 
    ("Robert","Sales","CA",81000,30,23000), 
    ("Maria","Finance","CA",90000,24,23000) 
  ]

columns= ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

simpleData2 = [
    ("James","Sales","NY",90000,34,10000), 
    ("Maria","Finance","CA",90000,24,23000), 
    ("Jen","Finance","NY",79000,53,15000), 
    ("Jeff","Marketing","CA",80000,25,18000), 
    ("Kumar","Marketing","NY",91000,50,21000) 
  ]
columns2= ["employee_name", "department", "state", "salary", "age", "bonus"]

df2 = spark.createDataFrame(data=simpleData2, schema = columns2)

df2.printSchema()
df2.show(truncate=False)

unionDF = df.union(df2)
unionDF.show(truncate=False)
disDF = df.union(df2).distinct()
disDF.show(truncate=False)

unionAllDF = df.unionAll(df2)
unionAllDF.show(truncate=False)

## Union. Различный порядок колонок

In [ ]:
%pyspark
simpleData3 = [
    ("Sales","Jones","NY",90000,34,10000), 
    ("Finance","Maria","CA",90000,24,23000), 
    ("Finance","Jen","NY",79000,53,15000), 
  ]
columns3= ["department", "employee_name","state","salary","age","bonus"]

df3 = spark.createDataFrame(data = simpleData3, schema = columns3)

print("df:")
df.show()
print("df3:")
df3.show()

unionDF = df.union(df3)
unionDF.show(truncate=False)

## Safe union

In [ ]:
%pyspark

df.union(
    df3.select(df.columns)
).show(truncate=False)

## Regexp

In [ ]:
%pyspark

from pyspark.sql.functions import regexp_replace, col

habrData\
.select("link")\
.withColumn("company_id", regexp_replace(col("link"), "(https://habr.com/ru/company/)|(/blog/[0-9]+/)|(https://habr.com/ru/post/[0-9]+/)", "") )\
.show(50, False)

## Explode

In [ ]:
%pyspark

arrayArrayData = [
  ("James",[["Java","Scala","C++"],["Spark","Java"]]),
  ("Michael",[["Spark","Java","C++"],["Spark","Java"]]),
  ("Robert",[["CSharp","VB"],["Spark","Python"]])
]

df = spark.createDataFrame(data=arrayArrayData, schema = ['name','subjects'])
df.printSchema()
df.show(truncate=False)

from pyspark.sql.functions import explode
df2 = df.select(df.name,explode(df.subjects).alias("exploded"))
df2.show()

df2.select(df.name,explode(df2.exploded)).show(truncate=False)

df2.select(df.name,explode(df2.exploded)).printSchema()

In [ ]:
%pyspark
df.schema.json()

## Map to columns

In [ ]:
%pyspark

arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{})
        ]

df = spark.createDataFrame(data=arrayData, schema = ['name', 'knownLanguages', 'properties'])
df.printSchema()
df.show(truncate=False)

from pyspark.sql.functions import explode
df3 = df.select(df.name,explode(df.properties))
df3.printSchema()
df3.show()

## explode_outer

In [ ]:
%pyspark
from pyspark.sql.functions import explode_outer
""" with array """
df.select(df.name,explode_outer(df.knownLanguages)).show()
""" with map """
df.select(df.name,explode_outer(df.properties)).show()

## Window functions

### Create df for window examples

In [ ]:
%pyspark

simpleData = (("James", "Sales", 3000), 
    ("Michael", "Sales", 4600),  
    ("Robert", "Sales", 4100),   
    ("Maria", "Finance", 3000),  
    ("James", "Sales", 3000),    
    ("Scott", "Finance", 3300),  
    ("Jen", "Finance", 3900),    
    ("Jeff", "Marketing", 3000), 
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100) 
  )
 
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

In [ ]:
%pyspark

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, percent_rank, ntile, cume_dist, round
from pyspark.sql.types import FloatType

windowSpec  = Window.partitionBy().orderBy("salary")

df\
.withColumn("row_number", row_number().over(windowSpec)) \
.show(truncate=False)

### row_number, rank, dense_rank, percent_rank, ntile, cume_dist

In [ ]:
%pyspark

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, percent_rank, ntile, cume_dist, round
from pyspark.sql.types import FloatType

windowSpec  = Window.partitionBy("department").orderBy("salary")

df\
.withColumn("row_number", row_number().over(windowSpec)) \
.withColumn("rank", rank().over(windowSpec)) \
.withColumn("dense_rank", dense_rank().over(windowSpec)) \
.withColumn("percent_rank", percent_rank().over(windowSpec)) \
.withColumn("ntile", ntile(3).over(windowSpec)) \
.withColumn("cume_dist", round(cume_dist().over(windowSpec), 2 )) \
.show(truncate=False)

### lag Window Function

In [ ]:
%pyspark
from pyspark.sql.functions import lag, lead

df.withColumn("lag", lag("salary", 1).over(windowSpec)) \
.withColumn("lead", lead("salary", 2).over(windowSpec)) \
.show()

### Window Aggregate Functions

In [ ]:
%pyspark
from pyspark.sql.window import Window
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col,avg,sum,min,max,row_number

windowSpec     = Window.partitionBy("department").orderBy("salary")
windowSpecAgg  = Window.partitionBy("department")


df.withColumn("row", row_number().over(windowSpec)) \
.withColumn("avg_cum", avg(col("salary")).over(windowSpec)) \
.withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
.withColumn("sum_cum", sum(col("salary")).over(windowSpec)) \
.withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
.withColumn("min", min(col("salary")).over(windowSpecAgg)) \
.withColumn("max", max(col("salary")).over(windowSpecAgg)) \
.show()
# .where(col("row") == 1).select("department", "avg", "sum", "min", "max") \

In [ ]:
%pyspark

spark.sql("with () as a, select * from ")

## UDF

In [ ]:
%pyspark

habrData = spark.read.option("header", True).csv("/datasets/habr_data.csv")
 
from pyspark.sql.functions import udf, col, round
from pyspark.sql.types import IntegerType, FloatType
import pyspark.sql.functions as F

def mult(i):
    return i * 2

multUdf = udf(mult)

habrData \
.select("rating") \
.limit(10) \
.withColumn("udfString", multUdf(col("rating"))) \
.withColumn("udfInt", multUdf(col("rating").cast(IntegerType()))) \
.withColumn("round", round(col("udfInt")) ) \
.show()

In [ ]:
%pyspark

spark.createDataFrame(([1], [2], [3]), schema="n INT").show()

In [ ]:
%pyspark

spark.createDataFrame(([1], [2], [3]), schema="n INT") \
.selectExpr("n + 1") \
.explain()

In [ ]:
%pyspark

from pyspark.sql.functions import udf, col

def plusOne(i):
    return i + 1
    
plusOneUdf = udf(plusOne)

spark.createDataFrame(([1], [2], [3]), schema="n INT")\
.select(plusOneUdf(col("n")).alias("plusOneUdf"))\
.explain()

In [ ]:
%pyspark

spark.createDataFrame(([1], [2], [3]), schema="n INT")\
.selectExpr("n + 1 as plusOne")\
.where("plusOne = 2")\
.explain()

In [ ]:
%pyspark

spark.createDataFrame(([1], [2], [3]), schema="n INT")\
.withColumn("plusOne", plusOneUdf(col("n")))\
.where("plusOne = 2")\
.explain()

### Обработка null

In [ ]:
%pyspark

habrData = spark.read.option("header", True).csv("/datasets/habr_data.csv").cache()
 
from pyspark.sql.functions import udf, col, when, expr
from pyspark.sql.types import IntegerType

def mult(i):
    return i * 2
    
def mult_nullsafe(i):
    if i is None:
        return 0
    else:
        return i * 2
    
# multUdf = udf(mult) 
multUdf = udf(mult_nullsafe)

# .na.drop("all")\
habrData\
.select("rating")\
.withColumn("rating", col("rating").cast(IntegerType()))\
.withColumn("udfInt", multUdf(col("rating"))   )\
.show()

### SQL register UDF

In [ ]:
%pyspark

from pyspark.sql.functions import udf, col


spark.udf.register("mult_nullsafe", mult_nullsafe)

habrData \
.where("rating is not null") \
.withColumn("rating", col("rating").cast(IntegerType())) \
.createOrReplaceTempView("habr_data")

spark.sql("select rating, mult_nullsafe(rating) plus_one from habr_data").show()

### Lambda

In [ ]:
%pyspark

# Integer type output
from pyspark.sql.types import IntegerType

def square(i):
    if i is None:
        return 0
    else:
        return i * i
    
square_udf_int = udf(lambda z: square(z), IntegerType())

habrData\
.select("rating")\
.where("rating is not null")\
.withColumn("rating", col("rating").cast(IntegerType()))\
.withColumn("square", square_udf_int(col("rating"))   )\
.show()

### Обработка массивов

In [ ]:
%pyspark
from pyspark.sql.types import *

def square_list_float(x):
    return [float(val)**2 for val in x]


square_list_float_udf = udf(lambda y: square_list_float(y), ArrayType(FloatType()))


cSchema = StructType([StructField("int_array", ArrayType(IntegerType()))])

df = spark.createDataFrame(
    [[[1, 2]], [[3, 4, 5]], [[6, 7, 8, 9]]], schema=cSchema
)

df.printSchema()
df.show()

df\
.withColumn("square_list_float_udf", square_list_float_udf("int_array"))\
.show()

In [ ]:
%pyspark
print("https://arena-hadoop.inno.tech:18088/proxy/" + sc.applicationId)